In [16]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import pandas as pd
import scipy.sparse as sparse
import numpy as np
from scipy.sparse.linalg import spsolve
from tqdm.notebook import tqdm_notebook as tqdm


In [17]:
steam_raw = pd.read_csv("steam-200k.csv", header=None, names=["User_ID", "Game", "Interaction", "Hours", "Ignore"])

steam = steam_raw.drop("Ignore", axis =1)

In [18]:
steam_purchase = steam[steam['Interaction'] == "purchase"]
steam_play = steam[steam['Interaction'] == "play"]
steam_pp = pd.merge(steam_purchase, steam_play, how = 'left', left_on = ['User_ID', 'Game'], right_on = ['User_ID', 'Game'])
steam_pp['Hours_y'] = steam_pp['Hours_y'].fillna(0)

#put it all into a clean table
steam_clean = steam_pp.drop(['Interaction_x', 'Interaction_y', 'Hours_x'], axis = 1)
steam_clean.head()
#steam_clean.info()

,User_ID,Game,Hours_y
0,151603712,The Elder Scrolls V Skyrim,273.0
1,151603712,Fallout 4,87.0
2,151603712,Spore,14.9
3,151603712,Fallout New Vegas,12.1
4,151603712,Left 4 Dead 2,8.9


In [19]:
n_users = len(steam_clean.User_ID.unique())
n_games = len(steam_clean.Game.unique())

print('There are {0} users and {1} games in the data'.format(n_users, n_games))

steam_clean_rare = steam_clean.groupby('Game').filter(lambda x : len(x) > 10)

n_users = len(steam_clean_rare.User_ID.unique())
n_games = len(steam_clean_rare.Game.unique())

print('After removing rare games, there are {0} users and {1} games in the data'.format(n_users, n_games))

There are 12393 users and 5155 games in the data
After removing rare games, there are 12248 users and 1777 games in the data


In [20]:
steam_clean_rare['quartile_rating'] = steam_clean_rare.groupby(['Game'])['Hours_y'].rank(pct=True).transform(
    lambda x: pd.qcut(x, 5, labels=range(1,6), duplicates='drop')
)
steam_clean_rare

,User_ID,Game,Hours_y,quartile_rating
0,151603712,The Elder Scrolls V Skyrim,273.0,5
1,151603712,Fallout 4,87.0,5
2,151603712,Spore,14.9,5
3,151603712,Fallout New Vegas,12.1,3
4,151603712,Left 4 Dead 2,8.9,4
...,...,...,...,...
129529,128470551,Mortal Kombat Komplete Edition,2.5,3
129530,128470551,Fallen Earth,2.4,5
129531,128470551,Magic Duels,2.2,4
129533,128470551,Grand Theft Auto Vice City,1.5,4


In [21]:
# Create lists of all users, artists and plays
users = list(np.sort(steam_clean_rare.User_ID.unique()))
games = list(np.sort(steam_clean_rare.Game.unique()))
plays = list(steam_clean_rare.quartile_rating)

# Get the rows and columns for our new matrix
rows = pd.factorize(steam_clean_rare.User_ID.astype('category'))[0]
cols = pd.factorize(steam_clean_rare.Game.astype('category'))[0]

# Contruct a sparse matrix for our users and items containing number of plays
data_sparse = sparse.csr_matrix((plays, (rows, cols)), shape=(len(users), len(games)))

In [22]:
user_lookup = {r : u for r, u in zip(list(dict.fromkeys(rows)), steam_clean_rare.User_ID.unique())}
games_lookup = {c : g for c, g in zip(list(dict.fromkeys(cols)), steam_clean_rare.Game.unique())}

In [23]:
matrix_size = data_sparse.shape[0]*data_sparse.shape[1] # Number of possible interactions in the matrix
num_purchases = len(data_sparse.nonzero()[0]) # Number of items interacted with
sparsity = 100*(1 - (num_purchases/matrix_size))
sparsity


99.4591516463175

Maksymalna pustość macierzy dla algorytmu ALS wynosi 99.5%

In [24]:
import random
def make_train(ratings, pct_test = 0.2):
    '''
    This function will take in the original user-item matrix and "mask" a percentage of the original ratings where a
    user-item interaction has taken place for use as a test set. The test set will contain all of the original ratings, 
    while the training set replaces the specified percentage of them with a zero in the original ratings matrix. 
    
    parameters: 
    
    ratings - the original ratings matrix from which you want to generate a train/test set. Test is just a complete
    copy of the original set. This is in the form of a sparse csr_matrix. 
    
    pct_test - The percentage of user-item interactions where an interaction took place that you want to mask in the 
    training set for later comparison to the test set, which contains all of the original ratings. 
    
    returns:
    
    training_set - The altered version of the original data with a certain percentage of the user-item pairs 
    that originally had interaction set back to zero.
    
    test_set - A copy of the original ratings matrix, unaltered, so it can be used to see how the rank order 
    compares with the actual interactions.
    
    user_inds - From the randomly selected user-item indices, which user rows were altered in the training data.
    This will be necessary later when evaluating the performance via AUC.
    '''
    test_set = ratings.copy() # Make a copy of the original set to be the test set. 
    test_set[test_set != 0] = 1 # Store the test set as a binary preference matrix
    training_set = ratings.copy() # Make a copy of the original data we can alter as our training set. 
    nonzero_inds = training_set.nonzero() # Find the indices in the ratings data where an interaction exists
    nonzero_pairs = list(zip(nonzero_inds[0], nonzero_inds[1])) # Zip these pairs together of user,item index into list
    random.seed(0) # Set the random seed to zero for reproducibility
    num_samples = int(np.ceil(pct_test*len(nonzero_pairs))) # Round the number of samples needed to the nearest integer
    samples = random.sample(nonzero_pairs, num_samples) # Sample a random number of user-item pairs without replacement
    user_inds = [index[0] for index in samples] # Get the user row indices
    item_inds = [index[1] for index in samples] # Get the item column indices
    training_set[user_inds, item_inds] = 0 # Assign all of the randomly chosen user-item pairs to zero
    training_set.eliminate_zeros() # Get rid of zeros in sparse array storage after update to save space
    return training_set, test_set, list(set(user_inds)) # Output the unique list of user rows that were altered  


In [25]:
product_train, product_test, product_users_altered = make_train(data_sparse, pct_test = 0.2)


In [52]:
import implicit

alpha = 15
model = implicit.als.AlternatingLeastSquares(factors=20, regularization = 0.1, iterations = 50)
model.fit(product_train.T)

user_items = product_train.tocsr()

100%|██████████| 50/50 [00:08<00:00,  5.96it/s]


In [54]:
recommendations = model.recommend(1, user_items)

related = model.similar_items(2)
print(recommendations)
#print([x for x , _ in recommendations])
print(len(users))
print(len(games))


[(885, 0.04888074), (746, 0.034061037), (772, 0.02511097), (468, 0.020395521), (728, 0.017657027), (225, 0.013924815), (733, 0.013780642), (362, 0.012856776), (1028, 0.012627692), (520, 0.011444403)]
12248
1777
